<img src="./figures/rhdhv_logo.jpg" width=200 height=400 align="right" />

***

# Profile Optimizer (Bottom Level) DHydro - Jupyter notebook 


****

***

## Introduction
The **Profile Optimizer** is a Python tool in which the optimization of profiles for D-Hydro models is automated. Based on an existing D-Hydro model, part of the system will be modified to find an optimized situation. In this version (<font color='red'>v0.0.0-undecided</font>) it is possible to optimize the bottom level of profile of multiple branches based on the current bottom level and the slope. The bottom level is optimized to get higher water level during dry period while no flooding during storm events at the chosen location.

This notebook was set up during TKI5 as a workflow for the Pilot Aa of Weerijs-Profile Optimizer at waterboard Brabantse Delta.


## Contact 
The Profile Optimizer is part of HYDROLIB, an open source community for python tools for the D-Hydro software package. Visit the Hydrolib website for more information: https://github.com/Deltares/HYDROLIB

The Profile Optimizer is developed by Royal HaskoningDHV:
- <font color='red'>xxx</font>@rhdhv.com
- <font color='red'>xxx</font>@rhdhv.com
- <font color='red'>xxx</font>@rhdhv.com

****

### Content

* [Step 0: Prepare input](#step0)


* [Step 1: Create geometry and select area](#step1)
    * [Step 1.1: Create shapefiles for branches and cross section location](#step1.1)
    * [Step 1.2: Compute strahler order](#step1.2)
    * [Step 1.3: Select area](#step1.3)


* [Step 2: Spatial check](#step2)


* [Step 3: Optimize the bottom level](#step3)
    * [Step 3.1: Prepare optimizer](#step3.1)
    * [Step 3.2: Select reaches to be optimized](#step3.2)
    * [Step 3.3: Get constraints](#step3.3)
    * [Step 3.4: Get current water level](#step3.4)
    * [Step 3.5: Run optimization algorithm](#step3.5)


* [Limitations and suggestions](#step4)

*****
Import general packages:

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
from osgeo import ogr
from pathlib import Path
import time

In [ ]:
from geometry import create_branches, create_crosssections


***
### Step 0: Prepare input

In this step we select the model for which we want to use the Profile Optimizer (Bottom Level). From the selected model, a number of files are essential for this tool. These are identified in this step.

The work and output folder are also created.

### Folder

`geometry_folder`: the folder where the datasets of branches, crosssection_locations, selected_profiles, etc. created in the steps are saved. This folder may not yet exist.<br>
`spatialcheck_folder`: the folder where spatial checked model is written. This folder may not yet exist.<br>
`temp_folder`: the temporary folder where iterations are written. This folder must not yet exist and can be automatically deleted at the end of the process.<br>
`output_folder`: the folder where the optimized model is written. This folder must not yet exist.

In [ ]:
geometry_folder = 'C:/local/geometry'
spatialcheck_folder = 'C:/local/spatial_check'
temp_folder = 'C:/local/temp'
output_folder = 'C:/local/output'

### D-Hydro model input

`model_folder`: the folder where the source model is located (as for easy relative paths from here).<br>
`model_mdu`:  the MDU of this model.<br>
`model_network_nc`: the network file of this model.<br>
`model_crossloc`: the file containing the cross section locations of this model.<br>
`bat_file`: the batch file with which the DIMR calculation is performed.

In [ ]:
model_folder = Path(r'c:\Users\922383\Desktop\06_profile_optimizer_bottom_level\src\fm_modify')
model_mdu = model_folder/'Iteration_0.mdu'
model_network_nc = model_folder/'test_9_aa_of_weerijs_net.nc'
model_crossloc = model_folder/'crsloc.ini'
bat_file = model_folder/'run_bat.bat'

### Optimization area

`shapefile_path`shapefile (polygon) that selects the optimization area.

In [ ]:
shapefile_path = r'c:\Users\922383\Desktop\06_profile_optimizer_bottom_level\src\optimization_area_multireach.gpkg'

***
### Step 1: Create geometry and select area

In this step we create shapefiles for branches and cross section location based on network file and cross section location file from the model. Then, the strahler order will be computed using branches dataset. Last, we select part of the model for which we are going to do optimization. An example of branches and crosssection locations created are shown below. Note that strahler order 0 are bypasses which we will not take into consideration to optimize. 

<img src="./figures/geometry.jpg" width=600 />

#### Step 1.1: Create geometry dataset for branches and cross section location

In [ ]:
#from geometry import create_branches, create_crosssections
branches = create_branches(model_network_nc, geometry_folder)
#crosssection_locations = create_crosssections(branches, model_crossloc, geometry_folder)

In [ ]:
crosssection_locations = create_crosssections(branches, model_crossloc, geometry_folder)

#### Step 1.2: Compute strahler order

We use strahler.py to compute strahler order based on branches created in Step 1.1. The script is credited to Dr. Cho (https://here.isnew.info/strahler-stream-order-in-python.html). However, it is not mandatory to use this script. The objective is to compute strahler order for all branches. It is also fine if you use other method. Note that if you will use this script, please always check in GIS application after computing strahler order before using, because this script sometimes can wrongly compute small branches. The computing result `branches_strahler` will be used in Step 2.

In [ ]:
from strahler import find_head_lines, find_prev_lines, find_next_line, find_sibling_line

branches_shapefile = f'{geometry_folder}/branches.shp'
driver = ogr.GetDriverByName('ESRI Shapefile')
file = driver.Open(branches_shapefile, 1)
lyr = file.GetLayer(0)
num_features = lyr.GetFeatureCount()
lines = []
field_name = "Strahler"
for i in range(num_features):
        feat = lyr.GetFeature(i)
        feat.SetField(field_name, 0)
        lyr.SetFeature(feat)
        geom = feat.geometry()
        line = geom.GetPoints()
        lines.append(line)
    
head_idx = find_head_lines(lines)

for idx in head_idx:
    curr_idx = idx
    curr_feat = lyr.GetFeature(curr_idx)
    curr_ord = 1
    curr_feat.SetField(field_name, curr_ord) # head line always 1
    lyr.SetFeature(curr_feat)    

    while True:
        next_idx = find_next_line(curr_idx, lines)
        if not next_idx:
            break
        next_feat = lyr.GetFeature(next_idx)
        next_ord = next_feat.GetField(field_name)

        sibl_idx = find_sibling_line(curr_idx, lines)
        if sibl_idx is not None:
            sibl_feat = lyr.GetFeature(sibl_idx)
            sibl_ord = sibl_feat.GetField(field_name)
            
            if sibl_ord:
                if sibl_ord > curr_ord:
                    break
                elif sibl_ord < curr_ord:
                    if next_ord == curr_ord:
                        break
                else:
                    curr_ord += 1

        next_feat.SetField(field_name, curr_ord)
        lyr.SetFeature(next_feat)

        curr_idx = next_idx
            
# export lyr as .shp
driver = ogr.GetDriverByName('ESRI Shapefile')
out_ds = driver.CreateDataSource(f'{geometry_folder}/branches_strahler.shp')
out_layer = out_ds.CopyLayer(lyr, 'branches_strahler')
del out_layer, out_ds
print(f'Finished computing Strahler. The file is here: {geometry_folder}/branches_strahler.shp')
branches_strahler = gpd.read_file(f'{geometry_folder}/branches_strahler.shp')

#### Step 1.3: Select area

Note that in this version of the Profile Optimizer (Bottom Level), you can select multiple branches with multiple polygons in one shapefile. But you need to select all profiles within these branches. And the 'max allowable WL' (max allowable water level) should be specified in the attribute table of shapefile. If there are multiple polygons, the 'max allowable WL' should be the same. Otherwise only the first value of 'max allowable WL' in the shapefile will be used. See the example below.

<img src="./figures/select_area.jpg" width=500 />

In [ ]:
from geometry import select_crosssection_locations
selected_profiles = select_crosssection_locations(crosssection_locations, shapefile_path, geometry_folder)
selected_profiles.head()


***
### Step 2: Spatial check

In this step we will carry out spatial check for selected profile at joints and within each reach to make sure that the bottom levels does not increase from upstream to downstream. Key steps including preprocessing, route, spatial check at joints and within reaches, fill depression, and save the modified model. The saved model `spatial_check_mdu_path` will be used as the base model in Step 3. An example of profiles before and after spatial check is shown below.

<img src="./figures/spatial_check.jpg" width=900 />

In [ ]:
model_mdu

In [ ]:
from spatial_check import SpatialCheck
optimizer = SpatialCheck(base_model_fn=model_mdu, 
                         output_dir=spatialcheck_folder, 
                         selected_cross_loc=selected_profiles, 
                         branches=branches_strahler)

selectie = optimizer.preprocessing()
selectie_route = optimizer.route(selectie)
selectie_checked = optimizer.spatial_check(selectie_route) 
selectie_checked.to_csv(f'{geometry_folder}/selectie_checked.csv') # save to csv in geometry_folder
cross_def_new = optimizer.fill_depression(selectie_checked)
spatial_check_mdu_path = optimizer.save_model(cross_def_new)
spatial_check_mdu_path

***
### Step 3: Optimize the bottom level

The bottom level is optimized to get higher water level during dry period while no flooding during storm events in select area. 

There are two constraints when increasing the bottom level. One is the 'overcapacity', which is the difference between 'max allowable WL' and current water level of branches. The other is the constraint at joint, which means that, at joints, the bottom level of downstream profile cannot be higher than that of upstream profile. The increment step size is determined based on these two constraints. A maximum increment step size of 0.1 meter is also included to make sure the bottom level does not increase too fast.

<img src="./figures/illustrate.jpg" width=500 />

The bottom levels of specified profiles will be increased the same amount in each iteration. After increasing the bottom levels, we run the modified model and get water level results in select area. If the water level does not exceed the 'max allowable WL' and total increment of bottom level does not exceed the constraint at joints, we then calculate a new increment step size and carry out another iteration. The model will iterately run untill it does not meet the constraints anymore. After finishing the iteration, the model with optimized profiles will be saved as a separate new model.

#### Step 3.1: Prepare optimizer

In [ ]:
from optimizer_bottom_level import ProfileOptimizer
optimizer = ProfileOptimizer(base_model_fn=spatial_check_mdu_path,
                             work_dir=temp_folder,
                             output_dir=output_folder,
                             bat_file = bat_file,
                             shapefile= shapefile_path,
                             selectie_checked=selectie_checked,
                             iteration_name='Iteration')

#### Step 3.2: Select reaches to be optimized

Please check 'selectie_checked' dataframe, specify the reachid or reaches with certain strahler orders in the function below as list for optimization. If no reachid or strahler is specified, all profiles in the select area will be optimized.

In [ ]:
selectie_checked

In [ ]:
ids = optimizer.optimize_reach(reachid=[], strahler=[])
ids

#### Step 3.3: Get constraints

In [ ]:
# 'max allowable Wl' from shapefile_path
constraints = optimizer.get_constraints()
print(f'Max allowable water level = {constraints}')
# check and get constraints at joints
constraints2 = optimizer.constraint_at_joints(ids)  

#### Step 3.4: Get current water level

In this step, we will run original model and get initial water levels

In [ ]:
increment = 0
cross_def_new = optimizer.increase_bottom(ids, increment, output_folder=False)
iteration_folder = optimizer.create_iteration(cross_def_new)
print(iteration_folder)
optimizer.run_model(optimizer.work_dir/'run_bat.bat', iteration_folder)
selected_water_level = optimizer.get_water_level()

#### Step 3.5: Run optimization algorithm

In [ ]:
tot_increse = 0
# create iteration
while (constraints - selected_water_level['WL'].max())>= 0.01 and (not tot_increse>constraints2):  # the accuracy (0.01) can be decided by user
    if constraints2 == 0:
        print('WARNING: cannot increase this reach bottom level, because at joints bottom level at downstream is already the same as upstream. Or you can increase both at same time')
    else:
        start_time = time.time()  # time each iteration
        optimizer.iteration_nr += 1
        increment = optimizer.increment_step(selected_water_level, constraints, constraints2)
        cross_def_new = optimizer.increase_bottom(ids, increment, output_folder=False)
        iteration_folder = optimizer.create_iteration(cross_def_new)
        print(iteration_folder)
        tot_increse += increment    
        print('increment in this iteration: ', increment)
        optimizer.run_model(optimizer.work_dir/'run_bat.bat',iteration_folder)
        selected_water_level = optimizer.get_water_level()
        print('max water level: ', selected_water_level['WL'].max())
        print('total increse of bottom level: ', tot_increse)
        end_time = time.time()  # time each iteration
        print('time taken for this iteration:', end_time - start_time)  # print out the time taken for each iteration
print('Iteration stops')

In [ ]:
# export and save the modified model
optimizer.run_latest()
optimizer.export_model(specific_iteration='latest', cleanup=True)

***
### Limitations and suggestions

This is the first version of Profile Optimizer for Bottom Level. There are still a lot to improve. Here, I will discuss some main limitations of this version, and provide suggestions for future works.
***
**1. Type of water systems**<br>
In this version, we focus and develop the profile optimizer based on "natural drainage" systems which has upstream and downstream. This version is not applicable to polder systems, especially the "spatial_check" which requires flow in one direction following bottom level slope. The "Optimizer_bottom_level" might be applicable to polder systems, but needs to be modified and tested.
***
**2. Spatial_check**<br>
First, in this version, it is required that there is only one flow direction within one reach in select area. Because the "route" function needs to find the single most downstream profile. Therefore, it is suggested to check the model in DHydro software interface before select area to make sure the "correct" flow direction.

Second, when "fill_depression" after finding the depression profiles, the bottom levels of these depression profiles are only raised to the same level as their closest downstream profile. This may generate stair-case like bottoms. For future works, we suggest to carry out interpolation between the bottom levels of upstream and downstream of the depression profiles to get more smooth bottom.
***
**3. Optimizer_bottom_level**<br>
First, in this version, the select area can only allow the same constraint "max allowable WL" for all branches, which of course is not the case in reality. For future works, the scripts should be modified so that branches with different "max allowable WL" constraints are allowed in select area.

Second, the "max allowable WL" constraint for the whole catchment has not yet been defined. Only the constraint in select area is defined. In this case, during optimization, only the water levels of select area are checked. Water levels outside the select area are not checked. In the future, it is suggested that the "max allowable WL" constraint for the whole catchment should be defined in advance, either defined in polygons shapefile or in branches dataset. And the scripts should be modified to check all water levels in the Aa of Weerijs model during optimization.
***
**4. Structures**<br>
The structures within branches are not modified in this version. For example, the inlet and outlet level of culvert are not modified accordingly to the changes of profile bottom levels. This can influence the water level modeled. Therefore, in the future, the structures need to be taken into account when modifying profiles.